In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#Nous avons besoin de deux méthodes séparées pour les données de la NBA et des jeux olympiques
#puisque les pages HTML ne sont pas encodées de la même façon.
def ObtenirResultatsNBA(year):
    
    #On injecte l'année dans l'URL du site web pour avoir l'année souhaité
    url = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'.format(year)
    
    #On créer la requête HTTP et on parse le HTML grâce à la librairie BeautifulSoup
    r = requests.get(url)
    r_html = r.text
    soup = BeautifulSoup(r_html,'html.parser')
    
    #On trouve le thead du tableau afin d'obtenir les noms de colonne
    head = soup.find(class_="thead")
    #Nous trouvons l'élément HTML ayant comme class full_table, afin d'obtenir
    #le contenu du tableau
    body = soup.find_all(class_="full_table")
    
    #On nettoie le HTML reçu et on enlève les tags de padding au debut et à la fin
    nomscolonne=[head.text for item in head][0].replace("\n",",").split(",")[1:-1]
    
    joueurs=[]
    
    #On itère dans toutes les lignes de contenu du tableau
    for i in range(len(body)):
        
        joueur = []
        
        #On itère dans toutes les colonnes de la ligne en cours
        #On va chercher toutes les éléments ayant pour tag th ou td. Le rang 
        #th comme tag alors que les autres colonne sont td.
        for td in body[i].find_all(['th', 'td']):
            joueur.append(td.text)
    
        joueurs.append(joueur)
        
    #On créer un nouveau dataframe avec la liste des joueurs et les noms de colonne.
    df = pd.DataFrame(joueurs, columns=nomscolonne).set_index("Player")
    #On nettoie les noms des joueurs des charactères spéciaux pouvant s'y trouver
    df.index = df.index.str.replace('*', '', regex=True)
    
    return df

def ObtenirResultatsJO(year):
    
    #On injecte l'année dans l'URL du site web pour avoir l'année souhaité
    url = 'https://www.basketball-reference.com/international/mens-olympics/{}_per_game.html'.format(year)

    #On créer la requête HTTP et on parse le HTML grâce à la librairie BeautifulSoup
    r = requests.get(url)
    r_html = r.text
    soup = BeautifulSoup(r_html,'html.parser')
    
    #On va trouvé l'élément HTML ayant commme class stats_table
    table = soup.find(class_="stats_table")
    #On cherche l'élément thead à l'intérieur de table afin de trouver les noms de colomne
    head = table.find('thead')
    #On cherche l'élément tbody à l'intérieur de table afin de trouver le contenu
    #du tableau et on créer une liste de tous les éléments tr s'y trouvant afin 
    #de créer un tableau avec les données brutes 
    body = table.find('tbody').find_all('tr')
    
    #On nettoie le HTML reçu et on enlève les tags de padding au debut et à la fin
    nomscolonne = [head.text for item in head][0].replace("\n",",").split(",")[2:-2]
    
    joueurs = []
    
    #On itère dans toutes les lignes de contenu du tableau
    for i in range(len(body)):
        
        joueur = []
        
        #On itère dans toutes les colonnes de la ligne en cours
        #On va chercher toutes les éléments ayant pour tag th ou td. Le nom de joueur a 
        #th comme tag alors que les autres colonne sont td.
        for td in body[i].find_all(['th', 'td']):
            joueur.append(td.text)
    
        joueurs.append(joueur)
        
    #On créer un nouveau dataframe avec la liste des joueurs et les noms de colonne.
    df = pd.DataFrame(joueurs, columns=nomscolonne).set_index("Player")
    #On nettoie les noms des joueurs des charactères spéciaux pouvant s'y trouver
    df.index = df.index.str.replace('*', '', regex=True)
    
    return df

ObtenirResultatsJO('2020')


In [ ]:
#Import de la librairie pandas
import pandas

#Lecture des fichiers de données en spécifiant le séparateur point-virgule ';'
equipeJO = pandas.read_csv("20162020_JO_Team.csv", sep=';')
joueurJO2016 = pandas.read_csv("2016_JO_Player.csv", sep=';')
joueurNBA2016 = pandas.read_csv("2016_NBA_Player.csv", sep=';')
joueurNBAWP2016 = pandas.read_csv("2016_NBA_Player_WP.csv", sep=';')
equipeNBA2016 = pandas.read_csv("2016_NBA_Team.csv", sep=';')
joueurJO2020 = pandas.read_csv("2020_JO_Player.csv", sep=';')
joueurNBA2020 = pandas.read_csv("2020_NBA_Player.csv", sep=';')
equipeNBA2020 = pandas.read_csv("2020_NBA_Team.csv", sep=';')

#print(notes.dtypes)
joueurNBA2016

Quel est la corrélation entre la participation aux Olympiques et le classements des joueurs dans la NBA?

In [ ]:
# jointure
#df_sales_full = df_sales.join(df_product.set_index("product_id"), on="product_id")

#print(joueurJO2016.columns)
joueur2016 = joueurJO2016.join(joueurNBA2016.where(joueurNBA2016['Rank'] <= 12).set_index('Name'), on="Name", lsuffix='JO', rsuffix='NBA', how='inner').sort_values(by=['Rank'])
joueur2020 = joueurJO2020.join(joueurNBA2020.where(joueurNBA2020['Rank'] <= 12).set_index('Name'), on="Name", lsuffix='JO', rsuffix='NBA', how='inner').sort_values(by=['Rank'])

#joueur2020.transpose()
joueur2016

Est-ce que les joueurs de la NBA ont une performance similaire moyenne en saison régulière comparé aux olympiques (Points, rebound, assist et block)?

Est-ce que la performance moyenne d'une équipe de la NBA est semblable à une équipe Olympique?

Est-ce que l'écart-type est similaire entre les joueurs d'une équipe de la NBA et ceux d'une équipe Olympique?

In [ ]:
#L'ecart-type des joueurs de l'equipe NBA 2016
joueurNBA2016[["Points Per game","Rebounds Per Game","Assists per game","Blocks per Game"]]
NBA2016= joueurNBA2016.agg({"Points Per game": ["std"],
           "Rebounds Per Game": ["std"],
           "Assists per game":["std"],
           "Blocks per Game": ["std"]})
NBA2016

In [ ]:
# L'écart-type des joueurs de l'équipe Olympique 2016
joueurJO2016[["Points Average","Rebounds Average","Assists Average","Average Blocks"]]
J02016= joueurJO2016.agg({"Points Average": ["std"],
           "Rebounds Average": ["std"],
           "Assists Average":["std"],
           "Average Blocks": ["std"]})
J02016

In [ ]:
# L'écart-type des joueurs de l'équipe Olympique 2020
joueurJO2020
joueurJO2020[["Points Average","Rebounds Average","Assists Average","Blocks Average"]]
Jo2020= joueurJO2020.agg({"Points Average": ["std"],
           "Rebounds Average": ["std"],
           "Assists Average":["std"],
           "Blocks Average": ["std"]})
JO2020

In [ ]:
#L'écart-type des joueurs de l'equipe NBA 2020
joueurNBA2020[["Points Per game","Rebounds Per Game","Assists per game","Blocks per Game"]]
NBA2020= joueurNBA2020.agg({"Points Per game": ["std"],
           "Rebounds Per Game": ["std"],
           "Assists per game":["std"],
           "Blocks per Game": ["std"]})
NBA2020

In [ ]:
df_tauxchange = pandas.concat([NBA2020,JO2020,JO2016,NBA2016])


### 